Imports

In [1]:
import os
import time
import json
import base64
import datetime
from typing import Optional, Dict, Any

try:
    import requests
except Exception:
    requests = None

import re

Configurações globais

In [ ]:
LOCAL_API_URL = "http://localhost:1234/v1/chat/completions"

# Tem de ser path absoluto (Alterar para suas pastas)
CUTOUTS_FOLDER = r"C:/Users/rodri/OneDrive - Unesp/Documentos/GitHub/BD2-Trabalho/CroppedPersonsHighRes" 
OUTPUT_JSON_FOLDER = r"C:/Users/rodri/OneDrive - Unesp/Documentos/GitHub/BD2-Trabalho/WithLLM/OutputJSONHighResQwen8B"

# Tempo máximo e quantidade máximas de vezes para tentar
TIMEOUT = 180
RETRIES = 3

PERSON_PROMPT = (
    "You are given a single image crop showing one person. "
    "Produce EXACTLY one JSON object and nothing else with the following fields:\n"
    '  "clip_caption": a short label-style caption (1-6 words, lower-case, no trailing punctuation),\n'
    '  "blip_description": a concise 1-2 sentence description of the person (clothing, color, action, pose, objects; avoid names or unverifiable facts).\n'
    "If unsure use neutral phrasing (e.g., 'appears to be'). Return only valid JSON."
)

IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".webp", ".bmp")

Funções de apoio

In [5]:
# Função para apenas para pegar timestamp
def now_iso():
    return datetime.datetime.now().isoformat()

# Função para codificar imagem para base64 (Formato aceito pelas LLMs)
def encode_image_to_data_url(image_path: str) -> Optional[str]:
    try:
        with open(image_path, "rb") as fh:
            b = fh.read()
        b64 = base64.b64encode(b).decode("ascii")

        ext = os.path.splitext(image_path)[1].lower()

        mime = {
            ".png": "image/png",
            ".webp": "image/webp",
            ".bmp": "image/bmp"
        }.get(ext, "image/jpeg")

        return f"data:{mime};base64,{b64}"
    except Exception as e:
        print("encode_image_to_data_url erro:", e)
        return None

# Função para enviar imagem para LLM como payload
def send_image_to_local_llm(prompt_text: str, image_path: str, api_url: str, timeout: int) -> Optional[Dict[str, Any]]:
    data_url = encode_image_to_data_url(image_path)
    if not data_url:
        return None

    payload = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt_text},
                    {"type": "input_image", "image_url": {"url": data_url}}
                ]
            }
        ]
    }

    try:
        resp = requests.post(api_url, json=payload, timeout=timeout)
    except Exception as e:
        return {"erro": f"request_failed: {e}"}

    text = resp.text

    try:
        resp.raise_for_status()
    except Exception:
        return {"text": text, "status_code": resp.status_code}

    try:
        return resp.json()
    except Exception:
        return {"text": text}

# Função para extrair da response da LLM o caption estilo CLIP e a descrição estilo BLIP (Fallback utilizado por extract_json_from_response)
def _extract_from_text(text: str) -> Optional[Dict[str, str]]:
    if not text:
        return None
    txt = text.strip()

    try:
        parsed = json.loads(txt)
        if isinstance(parsed, dict) and \
           "clip_caption" in parsed and "blip_description" in parsed:
            return parsed
    except Exception:
        pass

    m1 = re.search(r'"clip_caption"\s*:\s*"([^"]+)"', txt)
    m2 = re.search(r'"blip_description"\s*:\s*"([^"]+)"', txt)
    if m1 and m2:
        return {"clip_caption": m1.group(1), "blip_description": m2.group(1)}

    return None

# Função para primária para extrair o JSON com base na response
def extract_json_from_response(resp: Any) -> Optional[Dict[str, Any]]:
    if not resp:
        return None

    if isinstance(resp, dict):
        if "clip_caption" in resp and "blip_description" in resp:
            return resp

        if "choices" in resp:
            try:
                content = resp["choices"][0]["message"]["content"]
                if isinstance(content, str):
                    return _extract_from_text(content)
            except Exception:
                pass

        if "text" in resp:
            return _extract_from_text(resp["text"])

    if isinstance(resp, str):
        return _extract_from_text(resp)

    return None

Pipeline Principal

In [6]:
def process_cutouts_folder(cutouts_folder: str, output_json_folder: str, prompt_text: str, api_url: str, retries: int, timeout: int):
    if not os.path.isdir(cutouts_folder):
        raise FileNotFoundError(f"Folder Cutouts não encontrado: {cutouts_folder}")

    os.makedirs(output_json_folder, exist_ok=True)

    files = []
    for root, _, filenames in os.walk(cutouts_folder):
        for f in sorted(filenames):
            if os.path.splitext(f)[1].lower() in IMAGE_EXTS:
                files.append(os.path.join(root, f))

    if not files:
        print("Nenhuma imagem encontrada em", cutouts_folder)
        return

    total = len(files)
    print(f"Encontrou {total} imagens.")

    for idx, img_path in enumerate(files, start=1):
        base = os.path.basename(img_path)
        name, _ = os.path.splitext(base)

        print(f"[{idx}/{total}] {base}")

        out_json_path = os.path.join(output_json_folder, f"{name}.json")

        if os.path.exists(out_json_path):
            print(" -> Já existe, pulando.")
            continue

        success_resp = None

        for attempt in range(1, retries + 1):
            try:
                resp = send_image_to_local_llm(
                    prompt_text, img_path,
                    api_url=api_url, timeout=timeout
                )
                success_resp = resp
                break
            except Exception as e:
                print(f" tentativa {attempt} falhou:", e)
                time.sleep(attempt)

        result = {
            "source_image": os.path.abspath(img_path),
            "timestamp": now_iso(),
            "lm_raw": success_resp,
            "lm_parsed": None,
            "error": None
        }

        if success_resp is None:
            result["error"] = "all attempts failed"
        else:
            parsed = extract_json_from_response(success_resp)
            if parsed:
                result["lm_parsed"] = parsed
                print(" -> Parsed JSON OK.")
            else:
                print(" -> Não pode parse JSON.")

        with open(out_json_path, "w", encoding="utf-8") as fh:
            json.dump(result, fh, ensure_ascii=False, indent=2)
        print(" -> Salvo:", out_json_path)

    print("Pronto.")

Main


In [7]:
if __name__ == "__main__":
    process_cutouts_folder(
        cutouts_folder=CUTOUTS_FOLDER,
        output_json_folder=OUTPUT_JSON_FOLDER,
        prompt_text=PERSON_PROMPT,
        api_url=LOCAL_API_URL,
        retries=RETRIES,
        timeout=TIMEOUT
    )

Encontrou 9 imagens.
[1/9] cam1_trk1_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[2/9] cam1_trk2_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[3/9] cam1_trk3_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[4/9] cam1_trk4_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[5/9] cam1_trk5_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[6/9] cam1_trk6_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[7/9] cam1_trk7_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[8/9] cam1_trk8_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[9/9] cam1_trk9_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
Pronto.


Prompt em Português (Ficou pior)

In [8]:
LOCAL_API_URL = "http://localhost:1234/v1/chat/completions"

# Tem de ser path absoluto (Alterar para suas pastas)
CUTOUTS_FOLDER = r"C:/Users/rodri/OneDrive - Unesp/Documentos/GitHub/BD2-Trabalho/CroppedPersonsHighRes"
OUTPUT_JSON_FOLDER = r"C:/Users/rodri/OneDrive - Unesp/Documentos/GitHub/BD2-Trabalho/WithLLM/OutputJSONHighResQwen8BPortugues"

# Tempo máximo e quantidade máximas de vezes para tentar
TIMEOUT = 180
RETRIES = 3

PERSON_PROMPT = (
    "Você receberá um recorte de imagem contendo exatamente uma pessoa. "
    "Produza EXATAMENTE um único objeto JSON, e nada além disso, contendo os seguintes campos:\n"
    '  "clip_caption": uma legenda curta em estilo de rótulo (1 a 6 palavras, todas em minúsculas, sem pontuação final),\n'
    '  "blip_description": uma descrição concisa em 1 a 2 frases sobre a pessoa (roupas, cores, ação, postura, objetos; evite nomes ou fatos não verificáveis).\n'
    "Se houver incerteza, use formulações neutras (por exemplo, 'aparenta estar'). "
    "Retorne apenas um JSON válido."
)

IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".webp", ".bmp")

Main

In [9]:
if __name__ == "__main__":
    process_cutouts_folder(
        cutouts_folder=CUTOUTS_FOLDER,
        output_json_folder=OUTPUT_JSON_FOLDER,
        prompt_text=PERSON_PROMPT,
        api_url=LOCAL_API_URL,
        retries=RETRIES,
        timeout=TIMEOUT
    )

Encontrou 9 imagens.
[1/9] cam1_trk1_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[2/9] cam1_trk2_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[3/9] cam1_trk3_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[4/9] cam1_trk4_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[5/9] cam1_trk5_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[6/9] cam1_trk6_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[7/9] cam1_trk7_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[8/9] cam1_trk8_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
[9/9] cam1_trk9_pexels-hikaique-109919_20251113T091521.738215_first.jpg
 -> Já existe, pulando.
Pronto.
